<a href="https://colab.research.google.com/github/aidibo/AI-handy-ollama/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-ollama langchain langchain-community Pillow faiss-cpu

In [35]:
from langchain_community.llms import Ollama

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)


In [ ]:
!pip install langchain_chroma
!pip install chromadb
!pip install transformers -U

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [10]:
import chromadb
from chromadb.config import Settings

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="chroma_db2")

# Create or get a collection
collection = client.get_or_create_collection(name="my_collection2")

# Function to load text from a file and insert it into the Chroma database
def load_text_to_chroma(file_path):
    loader = WebBaseLoader(file_path)
    data = loader.load()

# Function to query the Chroma database
def query_chroma(query_text):
    results = collection.query(
        query_texts=[query_text],
        n_results=5
    )
    return results

# Load content from test.txt into ChromaDB
load_text_to_chroma('https://lilianweng.github.io/posts/2023-06-23-agent/')

# Example of querying the database
query_result = query_chroma("what's Sensory Memory?")
print("Query results:", query_result)


Query results: {'ids': [[]], 'embeddings': None, 'documents': [[]], 'uris': None, 'data': None, 'metadatas': [[]], 'distances': [[]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [21]:
# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
import os
import shutil
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

file_path = "https://lilianweng.github.io/posts/2023-06-23-agent/"
CHROMA_PATH="chroma_db"
DATA_PATH = "/tmp/ai-demo"


def main():
    generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    # loader = DirectoryLoader(DATA_PATH, glob="abc2.txt")
    # documents = loader.load()
    loader = WebBaseLoader(file_path)
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[4]
    print(document.page_content)
    print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    # if os.path.exists(CHROMA_PATH):
    #     shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    # embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    # embedding_function = SentenceTransformerEmbeddings(model_name="nomic-embed-text")
    db = Chroma.from_documents(
        chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH)
     docs = db.similarity_search(HuggingFaceEmbeddings(), query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    main()
    # query_chroma("歌中那位老黑奴经历了一生的苦难，家人都先他而去")

Split 1 documents into 231 chunks.
Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it
{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decompositi

<ipython-input-21-56ba44c1b2db>:60: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH


Saved 231 chunks to chroma_db.


In [31]:
def query_chroma(query_text):
    # Load the Chroma database
    embeddings = HuggingFaceEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)

    # Perform similarity search
    docs = db.similarity_search(query_text, k=10)
    for doc in docs:
        print("Page Content:", doc.page_content)
        #print(doc[0].page_content)

In [39]:
file_path = "https://lilianweng.github.io/posts/2023-06-23-agent/"
CHROMA_PATH="chroma_db"
DATA_PATH = "/tmp/ai-demo"
embeddings = HuggingFaceEmbeddings()


def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
     docs = db.similarity_search(query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    query_chroma("What are the approaches to Task Decomposition?")

<ipython-input-39-096946492d6f>:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Query results: [Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do s

In [42]:
query_str = "what's Sensory Memory?"
file_path = "https://lilianweng.github.io/posts/2023-06-23-agent/"
CHROMA_PATH="chroma_db"
DATA_PATH = "/tmp/ai-demo"
embeddings = HuggingFaceEmbeddings()


def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
     docs = db.similarity_search(query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    query_chroma(query_str)

<ipython-input-42-b28e96f1ed40>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


Sensory Memory: This is the earliest stage of memory, providing the ability to retain impressions of sensory information (visual, auditory, etc) after the original stimuli have ended. Sensory memory typically only lasts for up to a few seconds. Subcategories include iconic memory (visual), echoic
Query results: [Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\n

# formal LLM

In [6]:
# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
import os
import shutil
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import WebBaseLoader

file_path = "https://lilianweng.github.io/posts/2023-06-23-agent/"
CHROMA_PATH="chroma_db_llm"
DATA_PATH = "/tmp/ai-demo-llm"


def main():
    generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    # loader = DirectoryLoader(DATA_PATH, glob="abc2.txt")
    # documents = loader.load()
    loader = WebBaseLoader(file_path)
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=10,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    #document = chunks[4]
    #print(document.page_content)
    #print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    # if os.path.exists(CHROMA_PATH):
    #     shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    # embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    # embedding_function = SentenceTransformerEmbeddings(model_name="nomic-embed-text")
    db = Chroma.from_documents(
        chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH)
     docs = db.similarity_search(HuggingFaceEmbeddings(), query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    main()

Split 1 documents into 131 chunks.


<ipython-input-6-33911072833d>:61: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
<ipython-input-6-33911072833d>:61: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub,

Saved 131 chunks to chroma_db_llm.


<ipython-input-6-33911072833d>:63: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [11]:
query_str = "what's Sensory Memory?"
CHROMA_PATH="chroma_db_llm"
DATA_PATH = "/tmp/ai-demo-llm"

embeddings = HuggingFaceEmbeddings()


def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
     docs = db.similarity_search(query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    query_chroma(query_str)

<ipython-input-11-ec416ea4850b>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


Sensory Memory: This is the earliest stage of memory, providing the ability to retain impressions of sensory information (visual, auditory, etc) after the original stimuli have ended. Sensory memory typically only lasts for up to a few seconds. Subcategories include iconic memory (visual), echoic memory (auditory), and haptic memory (touch).
Query results: [Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, e

# ---------------------start

In [14]:
# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
import os
import shutil
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import WebBaseLoader

file_path = "https://lilianweng.github.io/posts/2023-06-23-agent/"
CHROMA_PATH="chroma_db_llm"
DATA_PATH = "/tmp/ai-demo-llm"


def main():
    generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    # loader = DirectoryLoader(DATA_PATH, glob="abc2.txt")
    # documents = loader.load()
    loader = WebBaseLoader(file_path)
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=10,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    #document = chunks[4]
    #print(document.page_content)
    #print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    # if os.path.exists(CHROMA_PATH):
    #     shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    # embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    embedding_function = OllamaEmbeddings(model="nomic-embed-text")
    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH)
     docs = db.similarity_search(HuggingFaceEmbeddings(), query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    main()

Split 1 documents into 131 chunks.


ConnectError: [Errno 111] Connection refused

In [ ]:
# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
import os
import shutil
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings

query_str = "what's Sensory Memory?"
CHROMA_PATH="chroma_db_llm"
DATA_PATH = "/tmp/ai-demo-llm"

#embeddings = HuggingFaceEmbeddings()
embeddings = OllamaEmbeddings(base_url="http://127.0.0.1:11434",model="nomic-embed-text")


def query_chroma(query_text):
     db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
     docs = db.similarity_search(query_text)
     print(docs[0].page_content)
     print("Query results:", docs)

if __name__ == "__main__":
    query_chroma(query_str)